# Парсинг HTML, XPath

### выполнил - Колеганов Н.Д.

### Задание

Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru-новости. Для парсинга использовать XPath. Структура данных должна содержать:

    • название источника;

    • наименование новости;

    • ссылку на новость;

    • дата публикации.


## Загружаем необходимые библиотеки

In [0]:
import requests
import re
import pandas as pd
import json
from datetime import datetime
from lxml import html
from pprint import pprint

## Предварительные настройки

In [0]:
# создал датафрейм куда будем собирать найденную информацию
df = pd.DataFrame(columns=['source', 'news', 'linc', 'date'])

In [0]:
# указываем header
header = {
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/80.0.3987.87 Chrome/80.0.3987.87 Safari/537.36'
            }

## Функции для сбора

# MAIL.ru

In [0]:
def get_atr_mailru(u):
    global df
    req = requests.get(u, headers = header).text
    root = html.fromstring(req)
    source = 'mail.ru'
    news = root.xpath("//h1[@class='hdr__inner']")[0].text
    linc = u
    date = root.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime | //time[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    
    return [source,news,linc,date]
    

def get_news_mailru():
    global df
    url = 'https://mail.ru/?from=m'
    req = requests.get(url, headers = header).text
    
    root = html.fromstring(req)

    for item in root.xpath("//div[@class='list'][1]/a/@href"):
        print(item)
        df.loc[len(df)+1] = get_atr_mailru(item)
    print(df)

## LENTA.ru

In [0]:
def get_atr_lentaru(u):
    global df
    print(u)
    req = requests.get(u, headers = header).text
    root = html.fromstring(req)
    
    source = 'lenta.ru'
    news = root.xpath("//h1[@class='b-topic__title'] | //div[@class='premial-header__title'] | //h1[@class='brief-header__title']")[0].text
    linc = u
    try:
        date = root.xpath("//div[@class='b-topic__info']/time/@datetime | //div[@class='premial-header__date']")[0]
    except:
        date = "не определена"
    
    return [source,news,linc,date]
    

def get_news_lentaru():
    global df
    url = 'https://lenta.ru'
    req = requests.get(url, headers = header).text

    root = html.fromstring(req)

    for item in root.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item']/a/@href | //section[@class='b-layout js-layout b-layout_main']//div[@class='titles']//a/@href"):
        ext = re.findall('(https:\/\/[\w\. _\d]+)\/|\/(extlink)\/', item)

        if len(ext) > 0:
            r1 = root.xpath(f"//a[@href='{item}']/text()")
            d1 = root.xpath(f"//a[@href='{item}']/time/@datetime")
            try:
                df.loc[len(df)+1] = [ext[0],r1,item,d1]
            except:
                df.loc[len(df)+1] = [ext[0],"Надо поработать со сторонними сайтами",item, "Дата"]
        else:
            df.loc[len(df)+1] = get_atr_lentaru(url+ item)
    print(df)

# Результат

## Что собралось с MAIL.RU

In [6]:
get_news_mailru()

https://r.mail.ru/n322461667?&test_id=63&rnd=125477429
https://r.mail.ru/n322464854?&test_id=63&rnd=125477429
https://r.mail.ru/n322464463?&test_id=63&rnd=125477429
https://r.mail.ru/n322465710?&test_id=63&rnd=125477429
https://r.mail.ru/n322448667?&test_id=63&rnd=125477429
https://r.mail.ru/n322420554?&test_id=63&rnd=125359630
https://r.mail.ru/n322445650?&test_id=63&rnd=167809687
https://r.mail.ru/n322421154?&test_id=63&rnd=179337563
    source  ...                       date
1  mail.ru  ...  2020-03-18T21:31:00+03:00
2  mail.ru  ...  2020-03-18T22:35:09+03:00
3  mail.ru  ...  2020-03-18T22:24:44+03:00
4  mail.ru  ...  2020-03-18T22:57:27+03:00
5  mail.ru  ...  2020-03-18T16:14:29+03:00
6  mail.ru  ...  2020-03-17T21:00:31+03:00
7  mail.ru  ...  2020-03-17T10:43:00+03:00
8  mail.ru  ...  2020-03-18T08:00:23+03:00

[8 rows x 4 columns]


## Что собралось с LENTA.RU

In [10]:
get_news_lentaru()

https://lenta.ru/news/2020/03/18/fac/
https://lenta.ru/news/2020/03/18/visa/
https://lenta.ru/news/2020/03/18/data/
https://lenta.ru/news/2020/03/18/dnr/
https://lenta.ru/news/2020/03/18/medvedev/
https://lenta.ru/news/2020/03/18/trumppp/
https://lenta.ru/news/2020/03/18/alina/
https://lenta.ru/news/2020/03/18/frcor/
https://lenta.ru/news/2020/03/18/belarus/
https://lenta.ru/articles/2020/03/18/money/
https://lenta.ru/articles/2020/03/17/berov/
https://lenta.ru/brief/2020/03/18/virusision/
https://lenta.ru/news/2020/03/18/euroo/
https://lenta.ru/news/2020/03/18/who/
https://lenta.ru/brief/2020/03/16/chtozavirus/
https://lenta.ru/news/2020/03/18/cometogether/
https://lenta.ru/news/2020/03/18/journalisarrest/
https://lenta.ru/brief/2020/03/16/nepitaysya/
https://lenta.ru/news/2020/03/18/coronastras/
https://lenta.ru/news/2020/03/18/kritikachistogorazuma/
https://lenta.ru/news/2020/03/18/divorces/
https://lenta.ru/news/2020/03/18/kk/
https://lenta.ru/news/2020/03/18/pedofil2/
https://lent

# ОБЩИЙ РЕЗУЛЬТАТ В DataFrame

In [11]:
df.head(46)

,source,news,linc,date
1,mail.ru,В ВОЗ назвали хорошей готовность России к возм...,https://r.mail.ru/n322461667?&test_id=63&rnd=1...,2020-03-18T21:31:00+03:00
2,mail.ru,В Китае рассказали об эффективном лекарстве от...,https://r.mail.ru/n322464854?&test_id=63&rnd=1...,2020-03-18T22:35:09+03:00
3,mail.ru,В России обсудят введение нового режима занятости,https://r.mail.ru/n322464463?&test_id=63&rnd=1...,2020-03-18T22:24:44+03:00
4,mail.ru,ФАС не видит предпосылок для роста цен на бенз...,https://r.mail.ru/n322465710?&test_id=63&rnd=1...,2020-03-18T22:57:27+03:00
5,mail.ru,Назван срок жизни коронавируса на разных предм...,https://r.mail.ru/n322448667?&test_id=63&rnd=1...,2020-03-18T16:14:29+03:00
6,mail.ru,Власти выделят автопрому России 600 млн рублей,https://r.mail.ru/n322420554?&test_id=63&rnd=1...,2020-03-17T21:00:31+03:00
7,mail.ru,Как в России и СНГ спасаются от коронавируса (...,https://r.mail.ru/n322445650?&test_id=63&rnd=1...,2020-03-17T10:43:00+03:00
8,mail.ru,Где купить недорогую одежду для дома (на время...,https://r.mail.ru/n322421154?&test_id=63&rnd=1...,2020-03-18T08:00:23+03:00
9,lenta.ru,С плакатов о голосовании по Конституции убрали...,https://lenta.ru/news/2020/03/18/data/,2020-03-18T23:02:00+03:00
10,lenta.ru,ДНР закрыла границу с Украиной из-за коронавируса,https://lenta.ru/news/2020/03/18/dnr/,2020-03-18T22:43:00+03:00


## Еще вариант с LENTA.RU

# Функция сбора

In [0]:
def requests_to_lenta():
    main_link='https://lenta.ru/'
    req=requests.get(main_link, headers=header)
    root=html.fromstring(req.text)
    news_list=[]

    #Распарсим главную новость
    title=root.xpath("//div[@class='first-item']//h2/a/text()")
    if title:
        link=root.xpath("//div[@class='first-item']//h2/a/@href")
        news_dict=dict()
        news_dict['site']=main_link
        news_dict['title']=title[0].replace(u'\xa0', u' ')
        news_dict['link']=link[0]
        news_dict['time']=root.xpath("//div[@class='first-item']//h2/a/time/text()")
        news_list.append(news_dict)
    else:
        print('Main news not found on lenta.ru')

    #Распарсим остальные новости
    titles=root.xpath("//section[@class='row b-top7-for-main js-top-seven']"
                      "//div[@class='item']//a/text()")
    if len(titles)>0:
        links=root.xpath("//section[@class='row b-top7-for-main js-top-seven']"
                         "//div[@class='item']//a/@href")
        for i in range(len(links)):
            news_dict=dict()
            news_dict['site']=main_link
            news_dict['title']=titles[i].replace(u'\xa0', u' ')
            news_dict['link']=links[i]
            news_dict['time']=datetime.now().strftime("%d-%m-%Y %H:%M")
            news_list.append(news_dict)
    else:
        print('News not found on lenta.ru')

        #Выведем результат на экран
    pprint(news_list)

# Посмотрим что получилось

In [49]:
requests_to_lenta()

[{'link': '/news/2020/03/18/up/',
  'site': 'https://lenta.ru/',
  'time': ['22:58'],
  'title': 'В ВОЗ оценили готовность России к росту заразившихся '
           'коронавирусом'},
 {'link': '/news/2020/03/18/fac/',
  'site': 'https://lenta.ru/',
  'time': '18-03-2020 20:50',
  'title': 'ФАС прокомментировала удивление Путина росту цен на бензин'},
 {'link': '/news/2020/03/18/visa/',
  'site': 'https://lenta.ru/',
  'time': '18-03-2020 20:50',
  'title': 'США перестали выдавать визы в большинстве стран из-за '
           'коронавируса'},
 {'link': '/news/2020/03/18/data/',
  'site': 'https://lenta.ru/',
  'time': '18-03-2020 20:50',
  'title': 'С плакатов о голосовании по Конституции убрали дату'},
 {'link': '/news/2020/03/18/dnr/',
  'site': 'https://lenta.ru/',
  'time': '18-03-2020 20:50',
  'title': 'ДНР закроет границу с Украиной из-за коронавируса'},
 {'link': '/news/2020/03/18/medvedev/',
  'site': 'https://lenta.ru/',
  'time': '18-03-2020 20:50',
  'title': 'Медведев назвал п